- [Sentiment CLF - JAX/FLAX on TPUs + 🤗](https://www.kaggle.com/code/heyytanay/sentiment-clf-jax-flax-on-tpus-w-b)

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import jax
from jax import numpy as jnp, random
import numpy as np
import optax
from tqdm.notebook import tqdm
tqdm.pandas()
print("Using jax", jax.__version__)
print("Optax Version : {}".format(optax.__version__))
devices = jax.local_devices()
print(f"Found {len(devices)} devices.")
devices[0]

Using jax 0.3.13
Optax Version : 0.1.2
Found 8 devices.


TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)

In [6]:
import datasets
from datasets import load_dataset, load_metric
import flax

from itertools import chain
from typing import Callable

from flax.training.common_utils import get_metrics, onehot, shard, shard_prng_key
from flax.training import train_state
from flax import traverse_util

from transformers import AutoTokenizer, FlaxAutoModelForSequenceClassification, AutoConfig

jax.local_devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import datasets
from datasets import load_dataset, load_metric

import jax
import flax
import optax
import jaxlib
import jax.numpy as jnp

from itertools import chain
from typing import Callable

from flax.training.common_utils import get_metrics, onehot, shard, shard_prng_key
from flax.training import train_state
from flax import traverse_util

from transformers import AutoTokenizer, FlaxAutoModelForSequenceClassification, AutoConfig

jax.local_devices()

2022-06-20 21:28:03.877477: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-06-20 21:28:03.877519: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


ModuleNotFoundError: No module named 'transformers'

In [7]:
class Config:
    nb_epochs = 5
    lr = 2e-5
    per_device_bs = 32
    num_labels = 2
    model_name = 'bert-base-uncased'
    total_batch_size = per_device_bs * jax.local_device_count()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
CONFIG = dict(
    lr=2e-5,
    model_name = 'bert-base-uncased',
    epochs = 5,
    split = 0.10,
    per_device_bs = 32,
    seed = 42,
    num_labels = 2,
    infra = "Kaggle",
    competition = 'none',
    _wandb_kernel = 'tanaym'
)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [8]:
def simple_acc(preds, labels):
    assert len(preds) == len(labels), "Predictions and Labels matrices must be of same length"
    acc = (preds == labels).sum() / len(preds)
    return acc

class ACCURACY(datasets.Metric):
    def _info(self):
        return datasets.MetricInfo(
            description="Calculates Accuracy metric.",
            citation="TODO: _CITATION",
            inputs_description="_KWARGS_DESCRIPTION",
            features=datasets.Features({
                'predictions': datasets.Value('int64'),
                'references': datasets.Value('int64'),
            }),
            codebase_urls=[],
            reference_urls=[],
            format='numpy'
        )

    def _compute(self, predictions, references):
        return {"ACCURACY": simple_acc(predictions, references)}
    
metric = ACCURACY()

In [9]:
!ls /home/rflagg/data

training.1600000.processed.noemoticon.csv
training.1600000.processed.noemoticon.csv.zip


In [11]:
import pandas as pd

def split_and_save(file_path: str, split: float = 0.10):
    file = pd.read_csv(file_path, encoding='latin-1', names=['sentiment', 'id', 'date', 'query', 'username', 'text'])
    file = file[['sentiment', 'text']]
    file['sentiment'] = file['sentiment'].map({4: 1, 0: 0})
    
    file = file.sample(frac=1).reset_index(drop=True)
    split_nb = int(len(file) * split)
    
    test_set = file[:split_nb].reset_index(drop=True)
    train_set = file[split_nb:].reset_index(drop=True)
    
    train_set.to_csv("train_file.csv", index=None)
    test_set.to_csv("test_file.csv", index=None)
    print("Done.")

split_and_save("/home/rflagg/data/training.1600000.processed.noemoticon.csv")



Done.


In [12]:
# Get the training and testing files loaded in HF dataset format
raw_train = load_dataset("csv", data_files={'train': ['./train_file.csv']})
raw_test = load_dataset("csv", data_files={'test': ['./test_file.csv']})



Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/rflagg/.cache/huggingface/datasets/csv/default-f1e7f08119a1ffda/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/rflagg/.cache/huggingface/datasets/csv/default-ea959330fd406536/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
def preprocess_function(data):
    """
    Preprocessing function
    """
    texts = (data["text"],)
    processed = Config.tokenizer(*texts, padding="max_length", max_length=128, truncation=True)
    processed["labels"] = data["sentiment"]
    return processed


In [14]:
%%time
train_dataset = raw_train.map(preprocess_function, batched=True, remove_columns=raw_train["train"].column_names)
test_dataset = raw_test.map(preprocess_function, batched=True, remove_columns=raw_test['test'].column_names)


  0%|          | 0/1440 [00:00<?, ?ba/s]

  0%|          | 0/160 [00:00<?, ?ba/s]

CPU times: user 12min 28s, sys: 8min 59s, total: 21min 28s
Wall time: 1min 48s


In [15]:
train = train_dataset['train']
valid = test_dataset['test']
print(len(train), len(valid))

1440000 160000


In [16]:
config = AutoConfig.from_pretrained(Config.model_name, num_labels=Config.num_labels)
model = FlaxAutoModelForSequenceClassification.from_pretrained(Config.model_name, config=config, seed=42)

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing FlaxBertForSequenceClassification: {('cls', 'predictions', 'transform', 'LayerNorm', 'bias'), ('cls', 'predictions', 'bias'), ('cls', 'predictions', 'transform', 'dense', 'kernel'), ('cls', 'predictions', 'transform', 'dense', 'bias'), ('cls', 'predictions', 'transform', 'LayerNorm', 'scale')}
- This IS expected if you are initializing FlaxBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of FlaxBertForSequenceClassification were not initialized from the model checkpoint at bert-base-unca

In [17]:
num_train_steps = len(train) // Config.total_batch_size * Config.nb_epochs
learning_rate_function = optax.cosine_onecycle_schedule(transition_steps=num_train_steps, peak_value=Config.lr, pct_start=0.1)
print("The number of train steps (all the epochs) is", num_train_steps)



The number of train steps (all the epochs) is 28125


In [18]:
optimizer = optax.adamw(learning_rate=Config.lr, b1=0.9, b2=0.999, eps=1e-6, weight_decay=1e-2)

def loss_fn(logits, targets):
    loss = optax.softmax_cross_entropy(logits, onehot(targets, num_classes=Config.num_labels))
    return jnp.mean(loss)
def eval_fn(logits):
    return logits.argmax(-1)

class TrainState(train_state.TrainState):
    eval_function: Callable = flax.struct.field(pytree_node=False)
    loss_function: Callable = flax.struct.field(pytree_node=False)
        
state = TrainState.create(
    apply_fn = model.__call__,
    params = model.params,
    tx = optimizer,
    eval_function=eval_fn,
    loss_function=loss_fn,
)

def train_step(state, batch, dropout_rng):
    targets = batch.pop("labels")
    dropout_rng, new_dropout_rng = jax.random.split(dropout_rng)
    
    def loss_function(params):
        logits = state.apply_fn(**batch, params=params, dropout_rng=dropout_rng, train=True)[0]
        loss = state.loss_function(logits, targets)
        return loss
    
    grad_fn = jax.value_and_grad(loss_function)
    loss, grad = grad_fn(state.params)
    grad = jax.lax.pmean(grad, "batch")
    new_state = state.apply_gradients(grads=grad)
    metrics = jax.lax.pmean({'loss': loss, 'learning_rate': learning_rate_function(state.step)}, axis_name='batch')
    
    return new_state, metrics, new_dropout_rng

In [19]:
parallel_train_step = jax.pmap(train_step, axis_name="batch", donate_argnums=(0,))


In [20]:
def eval_step(state, batch):
    logits = state.apply_fn(**batch, params=state.params, train=False)[0]
    return state.eval_function(logits)


In [21]:
parallel_eval_step = jax.pmap(eval_step, axis_name="batch")


In [22]:
def sentimentTrainDataLoader(rng, dataset, batch_size):
    steps_per_epoch = len(dataset) // batch_size
    perms = jax.random.permutation(rng, len(dataset))
    perms = perms[: steps_per_epoch * batch_size]  # Skip incomplete batch.
    perms = perms.reshape((steps_per_epoch, batch_size))

    for perm in perms:
        batch = dataset[perm]
        batch = {k: jnp.array(v) for k, v in batch.items()}
        batch = shard(batch)

        yield batch

In [23]:
def sentimentEvalDataLoader(dataset, batch_size):
    for i in range(len(dataset) // batch_size):
        batch = dataset[i * batch_size : (i + 1) * batch_size]
        batch = {k: jnp.array(v) for k, v in batch.items()}
        batch = shard(batch)

        yield batch


In [24]:
state = flax.jax_utils.replicate(state)



In [25]:
rng = jax.random.PRNGKey(42)
dropout_rngs = jax.random.split(rng, jax.local_device_count())


In [ ]:
for i, epoch in enumerate(tqdm(range(1, Config.nb_epochs + 1), desc=f"Epoch...", position=0, leave=True)):
    rng, input_rng = jax.random.split(rng)

    # train
    with tqdm(total=len(train) // Config.total_batch_size, desc="Training...", leave=False) as progress_bar_train:
        for batch in sentimentTrainDataLoader(input_rng, train, Config.total_batch_size):
            state, train_metrics, dropout_rngs = parallel_train_step(state, batch, dropout_rngs)
            progress_bar_train.update(1)

    # evaluate
    with tqdm(total=len(valid) // Config.total_batch_size, desc="Evaluating...", leave=False) as progress_bar_eval:
        for batch in sentimentEvalDataLoader(valid, Config.total_batch_size):
            labels = batch.pop("labels")
            predictions = parallel_eval_step(state, batch)
            metric.add_batch(predictions=chain(*predictions), references=chain(*labels))
            progress_bar_eval.update(1)

    eval_metric = metric.compute()

    loss = round(flax.jax_utils.unreplicate(train_metrics)['loss'].item(), 3)
    eval_score = round(list(eval_metric.values())[0], 3)
    metric_name = list(eval_metric.keys())[0]
    
    print(f"{i+1}/{Config.nb_epochs} | Train loss: {loss} | Eval {metric_name}: {eval_score}")

Epoch...:   0%|          | 0/5 [00:00<?, ?it/s]

Training...:   0%|          | 0/5625 [00:00<?, ?it/s]